# Feature Generator :: COV

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.stats import skew, kurtosis, linregress
from matplotlib import pyplot as plt
import multiprocessing
from multiprocessing import Pool
import os
import math

In [2]:
# Path settings

path_var = 1 

if (path_var == 1):
    filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'

else:
    filepath = '/home/sharan/Desktop/Option_Pricing/datasets/'


In [3]:

clean = pd.read_csv(filepath + 'ttm_cleaned_BANKNIFTY_2014_2018_raw.csv')
clean = clean.dropna()
filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_BANKNIFTY_2015_2018.csv')
ts_data = pd.read_csv(filepath + 'INDEXVALUE_BANKNIFTY_2014_2018_combined_raw.csv')


# For NIFTY50

# clean = pd.read_csv(filepath + 'ttm_cleaned_NIFTY50_2014_2018_raw.csv')
# clean = clean.dropna()
# filt = pd.read_csv(filepath + 'ttm_filter_param_0.04_NIFTY50_2015_2018_new.csv')
# ts_data = pd.read_csv(filepath + 'INDEXVALUE_NIFTY50_2015_2018_combined_raw.csv')


In [4]:
filt['TTM']=filt['ttm']

In [5]:
#print(clean)
filt['Date'] = pd.to_datetime(filt['Date'])
filt = filt.sort_values(by='Date')
filt.reset_index(inplace = True, drop = True)
clean['Date'] = pd.to_datetime(clean['Date'])
clean['Expiry'] = pd.to_datetime(clean['Expiry'])
clean = clean.sort_values(by='Date')
clean.reset_index(inplace = True, drop = False)
ts_data['Date'] = pd.to_datetime(ts_data['Date'])  
ts_data = ts_data.sort_values(by='Date')
ts_data.reset_index(inplace = True, drop = False)

In [6]:
len(clean)

73855

In [7]:
def stats(data):
    return np.mean(data),np.std(data)

In [8]:
def spreadEMA(data):
    data['Spread']=data['Close']/data['Open']
    return np.mean(data['Spread'])

def jumpEMA(data):
    data['Jump']=data['Open']/data['Close'].shift(1)
    data=data.loc[1:,:]
    return np.mean(data['Jump'])

In [9]:
def atr_data(ext):
    '''
    This function is defined as 
    atr =  max[abs(High - Low)_t, (High_t - Close_t-1), (Low_t - Close_t-1)]
    '''
    # Function Logic
#     ext = pd.DataFrame()
#     temp = data.loc[np.logical_and(data.loc[:, 'Strike Price'] == strike , data.loc[:, 'Expiry']) == expiry]
#     ext = ext.append(temp, ignore_index = True)

    val1 = ext.loc[:, 'High']
    val2 = ext.loc[:, 'Low']
    val3 = ext.loc[:, 'Close']

    val = pd.DataFrame()
    val.insert(0, "ATR", 0)
    for i in range(1, len(ext)):
        low = val2.loc[i]
        high = val1.loc[i]
        prev_close = val3.loc[i-1]
        a = (high/low)
        
        if(high>prev_close):
            b = (high/prev_close)
        else:
            b = (prev_close/high)
            
        if(low>prev_close):
            c = (low/prev_close)
        else:
            c = (prev_close/low)            

        k = max(a, b, c)
        val.loc[i-1, "ATR"] = k
        
    return np.mean(val)[0]
       

### Option Data Functions

In [10]:
def ret(data, date, OHLC ,mode='log'):
    '''
    This function takes in 2 primary params. The function outputs a vector of simple return data
    corressponding to previous 20 days raw ts data.
    The function can also output the log returns on changing the value of the mode param to 'log'.

    ## Modification :: The function can also output just the raw rows if mode param is set to 'e'

    # Demo func
    ret(df, '31-Jan-2019')
    ret(df, '31-Jan-2019', 'log')
    ret(df, '31-Jan-2019', 'e')
    '''

    if (mode == 'log'): # Func logic if log returns is desired
        val = pd.DataFrame()
        temp = np.log(data[OHLC])-np.log(data[OHLC].shift(1))
        val['LR']=temp[1:]

    if (mode == 'e'): # Func logic to extract the raw data
        k = data.index[data['Date']==date][0]
        val = pd.DataFrame()
        val.insert(0, "Close", 0)
        val.insert(0, "High", 0)
        val.insert(0, "Low", 0)
        val.insert(0, "Date", 0)
        val["Close"] = data.loc[k-19:k, "Close"]
        val['High'] = data.loc[k-19:k, "High"]
        val['Low'] = data.loc[k-19:k, "Low"]
        val['Open'] = data.loc[k-19:k, "Open"]
        val['Date'] = data.loc[k-19:k, "Date"]
    val.reset_index(inplace=True, drop=True)
    return val

### Feature Generator

In [11]:
def gen(ts_data, ttm, atmDate, atmError, percentage, op_data, op_data_jump, strike,spot,opPrice, prevOp):
    
    oi = op_data['Open Int'].iloc[-1]
    #     print(oi)

    del_oi = op_data_jump['Change in OI'].iloc[-1]
    #     print(del_oi)

    change_oi = (del_oi/oi) *100

    ts_m,ts_sd = stats(ts_data['Close'])

    avg_atmError=ts_m/strike

    ts_atr = atr_data(ts_data)

    close_log_return = ret(ts_data, atmDate,'Close',mode = 'log')

    open_log_return = ret(ts_data, atmDate,'Open',mode = 'log')

    high_log_return = ret(ts_data, atmDate,'High',mode = 'log')

    low_log_return = ret(ts_data, atmDate,'Low',mode = 'log')    

    close_log_ret_m, close_log_ret_sd = stats(close_log_return['LR'])

    open_log_ret_m, open_log_ret_sd = stats(open_log_return['LR'])

    high_log_ret_m, high_log_ret_sd = stats(high_log_return['LR'])

    low_log_ret_m, low_log_ret_sd = stats(low_log_return['LR'])

    vcov = pd.DataFrame()

    vcov['close_lr']=close_log_return['LR']

    vcov['open_lr']=open_log_return['LR']

    vcov['high_lr']=high_log_return['LR']

    vcov['low_lr']=low_log_return['LR']

    covMatrix = vcov.cov()

    #     print(covMatrix)

    c=[]

    for i in range(4):
        for j in range(i+1,4):
            #             print(covMatrix.iloc[i,j])
            val = round(covMatrix.iloc[i,j],8)
            if(val>0):
                c.append(math.sqrt(val))
            else:
                c.append(-1*math.sqrt(abs(val)))

    return [atmDate] + [close_log_ret_m, close_log_ret_sd, 
    open_log_ret_m, open_log_ret_sd, high_log_ret_m, high_log_ret_sd, low_log_ret_m, 
    low_log_ret_sd] + c + [change_oi, avg_atmError,ts_atr,prevOp] + [ttm, atmError,strike,spot,opPrice,percentage]

In [12]:
cvList=[]
cov_list=['close_lr','open_lr','high_lr','low_lr']
for cl1 in range(len(cov_list)):
    for cl2 in range(cl1+1,len(cov_list)):
        cvl=cov_list[cl1]+"_"+cov_list[cl2]+"_cov"
        cvList.append(cvl)
# print(cvList)
col_name = ['date'] + ['close_mean','close_sd','open_mean','open_sd','high_mean','high_sd','low_mean','low_sd'] + cvList + ['change_oi','avg_atmError','ts_atr','prevOp'] + ['ttm', 'atmError','strike','spot','opPrice','percentage']

In [13]:
# col_name

In [14]:
len(filt)

32856

In [15]:
filt.reset_index(drop=True,inplace=True)
# filt

In [16]:
count = 0
# percentage = []

feat = []
params=[]
for index,row in filt.iterrows():
    ttm = row['TTM']
    strike=row['Strike Price']
    spot=row['Underlying Value']
    atmDate=row['Date']
    expDate=row['Expiry']
    close = row['Close']
    opPrice = close
    opsTS = clean.loc[np.logical_and(clean['Strike Price'] == strike, clean['Expiry'] == expDate)]
    opsTS=opsTS.drop("index",1)
    opsTS.reset_index(inplace=True, drop=True)
#     print(atmDate)
    ind = opsTS.index[opsTS['Date']==atmDate]#Gives Index of that day itself
    ind=ind[0]
    if (ind > 20):
        window = 20
    elif (ind < 1):
        continue
    else:
        window = ind
    
    percentage = (close/strike)*100
    
    ops=opsTS.loc[ind-window:ind-1,:]
    ops.reset_index(inplace=True, drop=True)
    if(window==20):
        adjWindow=19
    else:
        adjWindow=window
        
    opsJump=opsTS.loc[ind-adjWindow:ind,:]
    opsJump.reset_index(inplace=True, drop=True)
    ts = ret(ts_data, atmDate, '', mode = 'e')
    
    atmError=(spot/strike)
    prevOp=(opsTS.loc[ind-1,'Close']/strike)
    params.append((ts,ttm,atmDate,atmError,percentage,ops,opsJump,strike,spot,opPrice,prevOp))
#     print(opsJump.loc[:,['Open Int','Change in OI']])
#     count = count + 1
#     break
    
pool = Pool()
for res in pool.starmap(gen,params):    
    feat.append(res)
    
pool.close()


/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in double_scalars
  if __name__ == '__main__':
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/docboy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/home/

In [17]:
results=pd.DataFrame.from_records(feat, columns=col_name)
results

,date,close_mean,close_sd,open_mean,open_sd,high_mean,high_sd,low_mean,low_sd,close_lr_open_lr_cov,...,change_oi,avg_atmError,ts_atr,prevOp,ttm,atmError,strike,spot,opPrice,percentage
0,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,4.545455,1.009533,1.016899,0.042623,28.0,1.024615,18300.0,18750.45,768.40,4.198907
1,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,100.000000,0.987939,1.016899,0.032032,56.0,1.002698,18700.0,18750.45,750.00,4.010695
2,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,100.000000,0.972340,1.016899,0.027937,56.0,0.986866,19000.0,18750.45,610.00,3.210526
3,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,7.692308,0.947408,1.016899,0.017692,56.0,0.961562,19500.0,18750.45,334.65,1.716154
4,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,27.226463,0.952291,1.016899,0.010018,28.0,0.966518,19400.0,18750.45,189.05,0.974485
5,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,15.066470,0.967249,1.016899,0.015780,28.0,0.981699,19100.0,18750.45,300.95,1.575654
6,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,-0.714286,0.962211,1.016899,0.013625,28.0,0.976586,19200.0,18750.45,256.90,1.338021
7,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,1.031895,0.957226,1.016899,0.011762,28.0,0.971526,19300.0,18750.45,221.30,1.146632
8,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,17.045928,0.947408,1.016899,0.008379,28.0,0.961562,19500.0,18750.45,160.35,0.822308
9,2015-01-01,-0.000091,0.011315,-0.000086,0.012281,-0.000032,0.008066,0.000180,0.012087,0.005700,...,27.245509,0.982684,1.016899,0.024423,28.0,0.997364,18800.0,18750.45,445.15,2.367819


In [19]:
results.to_csv(filepath + 'BANKNIFTY_feature_set_COV_STEP_1.csv', index = False)



# results.to_csv(filepath + 'NIFTY50_feature_set_COV_STEP_1.csv', index = False)